In [1]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split 
from datetime import date

In [2]:
data = pd.read_csv('data/consumer_complaints.csv')


/home/harshit/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def f(x):    
    month_a, day_a, year_a = x['date_received'].split('/')
    month_b, day_b, year_b = x['date_sent_to_company'].split('/')
    date_a = date(year=int(year_a), month=int(month_a), day=int(day_a))
    date_b = date(year=int(year_b), month=int(month_b), day=int(day_b))

    return  (date_b- date_a).days+1
 

data['Time Allotted'] = data.apply(f, axis=1)

In [4]:
remove = ["consumer_complaint_narrative","complaint_id","date_sent_to_company","timely_response","company_public_response"]
data.drop(remove, axis = 1, inplace=True)

In [5]:
data['id'] = data.index
data.groupby(['company'])['id'].count().reset_index().sort_values('id', ascending = False)['company'].tolist()[0:10]

['Bank of America',
 'Wells Fargo & Company',
 'JPMorgan Chase & Co.',
 'Equifax',
 'Experian',
 'Citibank',
 'TransUnion Intermediate Holdings, Inc.',
 'Ocwen',
 'Capital One',
 'Nationstar Mortgage']

In [6]:
selected_companies = ['Bank of America',
 'Wells Fargo & Company',
 'JPMorgan Chase & Co.',
 'Equifax',
 'Experian',
 'Citibank',
 'TransUnion Intermediate Holdings, Inc.',
 'Ocwen',
 'Capital One',
 'Nationstar Mortgage']
data = data[data['company'].isin(selected_companies)]

In [7]:
data.groupby(['issue'])['id'].count().reset_index().sort_values('id', ascending = False)['issue'].tolist()[0:10]

['Loan modification,collection,foreclosure',
 'Incorrect information on credit report',
 'Loan servicing, payments, escrow account',
 'Account opening, closing, or management',
 "Credit reporting company's investigation",
 'Deposits and withdrawals',
 'Unable to get credit report/credit score',
 'Application, originator, mortgage broker',
 'Other',
 'Billing disputes']

In [8]:
selected_issues = ['Loan modification,collection,foreclosure',
 'Incorrect information on credit report',
 'Loan servicing, payments, escrow account',
 'Account opening, closing, or management',
 "Credit reporting company's investigation",
 'Deposits and withdrawals',
 'Unable to get credit report/credit score',
 'Application, originator, mortgage broker',
 'Other',
 'Billing disputes']
data = data[data['issue'].isin(selected_issues)]

In [9]:
data = data[pd.notna(data['state'])]
data = data[pd.notna(data['zipcode'])]

In [10]:
data.isna().mean()

date_received                   0.000000
product                         0.000000
sub_product                     0.408571
issue                           0.000000
sub_issue                       0.638131
company                         0.000000
state                           0.000000
zipcode                         0.000000
tags                            0.873193
consumer_consent_provided       0.811215
submitted_via                   0.000000
company_response_to_consumer    0.000000
consumer_disputed?              0.000000
Time Allotted                   0.000000
id                              0.000000
dtype: float64

In [11]:
def downsample(df,colname):
    classOne = df[df[colname]=='Closed without relief'].index
    classTwo = df[df[colname]=='Closed with monetary relief'].index
    arr = list(classOne)
    arr2 = list(classTwo)
    random.shuffle(arr)
    random.shuffle(arr2)
    toKeep = arr[0:1800]
    toKeep2 = arr2[0:1700]
    df_settlement_balanced = df[df.index.isin(toKeep+toKeep2)]
    return df_settlement_balanced.reset_index(drop=True)

df = downsample(data, 'company_response_to_consumer')
df.shape

(3500, 15)

In [12]:
df['id'] = df.index

In [13]:
df.isna().mean()

date_received                   0.000000
product                         0.000000
sub_product                     0.183143
issue                           0.000000
sub_issue                       0.989429
company                         0.000000
state                           0.000000
zipcode                         0.000000
tags                            0.881714
consumer_consent_provided       0.927143
submitted_via                   0.000000
company_response_to_consumer    0.000000
consumer_disputed?              0.000000
Time Allotted                   0.000000
id                              0.000000
dtype: float64

In [14]:
mapping = {
    "consumer_consent_provided":{
         'Consent not provided':'Customer does not demand compensation',
    'Consent provided':'Customer demand compensation'
    },
    "company_response_to_consumer":{
         'Closed with monetary relief':1,
    'Closed without relief':0
    }
    
}


df = df.replace(mapping)

In [15]:
df.columns = ["Date", "Product", "Sub product","Issue", "Sub issue", "Associated company", "State", "Zipcode", "Tags", "Customer demand", "Submitted via", "Compensation" ,"Past dispute", "Time alloted", "id"]

In [16]:
df = df[["id","Date", "Product", "Sub product","Issue", "Sub issue", "Associated company", "State", "Zipcode", "Tags", "Customer demand", "Submitted via" ,"Past dispute", "Time alloted", "Compensation"]]

In [17]:
df.groupby(['Compensation'])['id'].count().reset_index()

,Compensation,id
0,0,1800
1,1,1700


In [18]:
df

,id,Date,Product,Sub product,Issue,Sub issue,Associated company,State,Zipcode,Tags,Customer demand,Submitted via,Past dispute,Time alloted,Compensation
0,0,09/10/2013,Bank account or service,Savings account,Deposits and withdrawals,NaN,Citibank,CA,90651,NaN,NaN,Web,No,1,1
1,1,09/10/2013,Bank account or service,Checking account,Deposits and withdrawals,NaN,Bank of America,PA,19143,NaN,NaN,Phone,No,1,1
2,2,09/10/2013,Credit card,NaN,Other,NaN,Capital One,FL,34743,NaN,NaN,Postal mail,No,4,1
3,3,09/10/2013,Bank account or service,Checking account,Deposits and withdrawals,NaN,Bank of America,TN,37217,NaN,NaN,Referral,No,1,1
4,4,09/24/2013,Credit card,NaN,Billing disputes,NaN,Capital One,CA,94002,NaN,NaN,Web,No,1,1
5,5,08/31/2013,Credit card,NaN,Other,NaN,Citibank,CA,91367,NaN,NaN,Web,No,1,1
6,6,08/31/2013,Credit card,NaN,Billing disputes,NaN,Citibank,CA,92117,NaN,NaN,Web,No,1,1
7,7,09/02/2013,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,Wells Fargo & Company,TX,78258,NaN,NaN,Web,No,2,1
8,8,09/02/2013,Mortgage,Home equity loan or line of credit,"Loan servicing, payments, escrow account",NaN,JPMorgan Chase & Co.,TX,75006,Older American,NaN,Web,No,2,1
9,9,09/10/2013,Bank account or service,Savings account,Deposits and withdrawals,NaN,Citibank,IL,60680,Servicemember,NaN,Phone,Yes,2,1


In [21]:
# """Spliting datatset"""

train_mask = (df['Date'] >= '2011-01-01') & (df['Date'] <= '2014-12-31')
test_mask = (df['Date'] >= '2015-01-01') & (df['Date'] <= '2016-12-31')
train = df[train_mask]
test = df[test_mask]

In [22]:


for column in ["Product", "Associated company", "Issue", "State", "Submitted via", "Past dispute","Sub product"]:
    train= train[train[column].isin(test[column])]
    test= test[test[column].isin(train[column])]

train.to_csv("train.csv", index = False)
y_test = test[['id','Compensation']]
y_output = pd.DataFrame()
y_output['id'] = y_test['id'].tolist()
y_output['Compensation'] = [random.choice([0,1]) for i in range(y_test.shape[0])]
test.drop(['Compensation'], axis = 1, inplace = True)
test.to_csv("test.csv", index = False)
y_test.to_csv("actual_output.csv", index = False)
y_output.to_csv('sample_output.csv', index = False)

In [2]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [31]:
train.drop(['Sub issue'], axis = 1, inplace = True)
test.drop(['Sub issue'], axis = 1, inplace = True)

In [32]:
train.to_csv("train.csv", index = False)
test.to_csv("test.csv", index = False)